In [1]:
import pandas as pd

In [2]:
# Azure resources
subscription_id = "e478b470-4e14-4384-bc05-0e03fe0c2d9e"
resource_group = "HO_aml"  
workspace_name = "tooneaml"  
workspace_region = "Korea Central" 


In [16]:
from azureml.core import Workspace
ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      create_resource_group=True,
                      location=workspace_region
                     )

WorkspaceException: WorkspaceException:
	Message: Workspace with name 'tooneaml' already exists under resource group with name 'HO_aml'.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Workspace with name 'tooneaml' already exists under resource group with name 'HO_aml'."
    }
}

# 3. initialize workspace

In [2]:
from azureml.core import Workspace
ws = Workspace(subscription_id="e478b470-4e14-4384-bc05-0e03fe0c2d9e",
               resource_group="HO_aml",
               workspace_name="tooneaml")

In [5]:
ws = Workspace.from_config(path="../tooneaml_config.json")

# 4. experiments

In [3]:
from azureml.core.experiment import Experiment

# Choose an experiment name.
experiment_name = 'aischool-text-classification-exp-02'
experiment = Experiment(ws, experiment_name)


In [5]:
# 실험 리스트 확인
list_experiments = Experiment.list(ws)
for exp in list_experiments:
    print(exp.id, exp.name)

ab500e2c-7810-4d8c-b768-ecea37fae5a9 demo-040-exp
c3620040-5d78-4f5d-b8e6-0700585cacec demo-designer-01
3370e926-d7c1-44e9-90f2-931b262587bf demo-model-05-exp
86deaa5b-5287-4c05-be04-82c301b71287 explain_model
e4163005-e00b-43f9-9767-72285a517354 demo-06-automl-classification-text-dnn
8eef541f-cfc8-4131-902b-c0914ba3a506 automl-classification-text-dnn
ca168908-8075-4745-b89a-256d0cf3c5e4 demo-08-deploy-aks-automl-classification-bmarketing-all
e40698be-4d02-4697-8490-0b5a82a0e9d6 aischool-text-classification-exp-01


In [5]:
output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Subscription ID,e478b470-4e14-4384-bc05-0e03fe0c2d9e
Workspace Name,tooneaml
Resource Group,HO_aml
Location,koreacentral
Experiment Name,aischool-text-classification-exp-01


# 5. create remote compute target

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster.
amlcompute_cluster_name = "aischool-clu-01"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # CPU for BiLSTM, such as "STANDARD_D2_V2" 
                                                           # To use BERT (this is recommended for best performance), select a GPU such as "STANDARD_NC6" 
                                                           # or similar GPU option
                                                           # available in your workspace
                                                           max_nodes = 1)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# 5. runconfiguration, scriptrunconfig, run

run 객체는 실험을 제출하면서 생성

In [ ]:
# 현재 conda env 환경을 yml 파일로 export
conda env export > environment.yml
# import 
conda env create -f environment.yml

In [5]:
from azureml.core import ScriptRunConfig, RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

compute_config = RunConfiguration()
compute_config.target = compute_target
#compute_config.amlcompute.vm_size = "STANDARD_D1_V2"

conda_dep = CondaDependencies()
conda_dep.add_pip_package("scikit-learn")
conda_dep.add_pip_package("nltk")
conda_dep.add_pip_package("pandas")
conda_dep.add_pip_package("matplotlib")
conda_dep.add_pip_package("seaborn")
conda_dep.add_pip_package("beautifulsoup4")

conda_dep.add_pip_package("azureml-mlflow")



#conda_dep = CondaDependencies(conda_dependencies_file_path='./environment.yml', _underlying_structure=None)

compute_config.environment.python.conda_dependencies = conda_dep



In [30]:
from azureml.core import ScriptRunConfig, RunConfiguration
from azureml.core.environment import Environment

aienv = Environment.from_existing_conda_environment(name = "aienv", conda_environment_name = "toone_aml")

compute_config = RunConfiguration()

# Attach compute target to run config
compute_config.target = compute_target
# runconfig.run_config.target = "local"

# compute_config.amlcompute.vm_size = "STANDARD_D1_V2"


compute_config.environment = aienv

Exporting conda specifications for existing conda environment: toone_aml


In [38]:
run =  experiment.start_logging()

In [39]:
# run a trial from the train.py code in your current directory
src = ScriptRunConfig(source_directory='./', script='train.py', run_config=compute_config)

In [40]:
tags = {"dev": "phase-1-model-aischool"}
run = experiment.submit(config=src, tags=tags)

# Monitor a Run


In [37]:
run_details = run.get_details()
print(run_details)

{'runId': 'aischool-text-classification-exp-02_1588901167_3dadd2e1', 'target': 'aischool-clu-01', 'status': 'Queued', 'warnings': [{'message': "This compute target type doesn't support non-Docker runs; overriding run configuration to enable Docker.\nPlease enable Docker in the environment section of your run configuration to stop seeing this warning message."}], 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '919e07c2-8dae-4f13-94a6-d33850974ec9', 'azureml.git.repository_uri': 'https://github.com/holictoweb/holic_toone.git', 'mlflow.source.git.repoURL': 'https://github.com/holictoweb/holic_toone.git', 'azureml.git.branch': 'master', 'mlflow.source.git.branch': 'master', 'azureml.git.commit': '13388d62afb52dce7d7a9444069ccff3a2a626ea', 'mlflow.source.git.commit': '13388d62afb52dce7d7a9444069ccff3a2a626ea', 'azureml.git.dirty': 'True', 'AzureML.DerivedImageName': 'azureml/azureml_b20bf64ece196cc294d2ddd6a809d543', 'ProcessInfoFile': 'azureml-logs/process_

In [41]:
from azureml.widgets import RunDetails  
RunDetails(run).show()


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [29]:
run.complete()

# See the Results

In [30]:
run.wait_for_completion(show_output=False)

# now there is a trained model on the remote cluster
print(run.get_metrics())


{'mse': 0.10674668121476633}


# register the Model

In [45]:
model = run.register_model(model_name = "aischool_model", model_path = "/outputs/aischool_model.pkl", tags={'area': 'mnist'})

In [46]:
# model check
print("Registered model:\n --> Name: {}\n --> Version: {}\n --> URL: {}".format(model.name, model.version, model.url))

Registered model:
 --> Name: aischool_model
 --> Version: 1
 --> URL: aml://asset/681243cafa9c41b087a997a8823da344


# ========================================================================
# deploy


1. create env

In [ ]:
from azureml.core import Environment


#user_managed_env.python.user_managed_dependencies = True

# 구성되어 있는 conda 환경으로 만들기
aisenv = Environment.from_existing_conda_environment(name = "toone_aml",
                                                    conda_environment_name = "toone_aml")

# You can choose a specific Python environment by pointing to a Python path 
#user_managed_env.python.interpreter_path = '/home/johndoe/miniconda3/envs/myenv/bin/python'

aisenv.register(workspace=ws)

In [ ]:
2. scoring script